In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

update_line_webhook(webhook_url)

Ngrok URL: https://whistleable-lupe-perfunctory.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://whistleable-lupe-perfunctory.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Ming Hsin University of Science and Technology, MHUST）是一所位於台灣新竹縣的私立科技大學，以其悠久的歷史、實務導向的教學和卓越的產學合作而聞名。

**以下是明新科技大學的簡要介紹：**

1.  **創校歷史與發展：**
    *   創立於**1966年**，原名為「明新工業專科學校」。
    *   是台灣第一批由專科學校升格為科技大學的技職院校之一，於**1997年**改制為「明新技術學院」，並在**2002年**正式升格為「明新科技大學」。
    *   擁有超過半世紀的辦學經驗，是桃竹苗地區歷史最悠久的私立科技大學。

2.  **地理位置優勢：**
    *   學校地理位置優越，毗鄰**新竹科學園區**及多個工業區，這為學生提供了豐富的實習、就業和產學合作機會。

3.  **教育理念與特色：**
    *   秉持「**誠信、勤奮、創新、卓越**」的校訓，致力於培養具備專業技能、實務能力及人文素養的產業人才。
    *   **實務導向教學：** 注重理論與實務結合，強調動手實作，課程設計緊密連結產業需求。
    *   **產學合作：** 與企業界建立深厚的合作關係，提供學生校外實習、專題製作及就業輔導，畢業生深受企業肯定，就業率高。

4.  **學術組織與系所：**
    *   設有四大學院：
        *   **工程學院：** 涵蓋電子、電機、機械、土木、資訊工程等領域，是其傳統強項。
        *   **管理學院：** 提供企業管理、資訊管理、行銷與流通管理、財務金融等課程。
        *   **服務事業學院：** 包含旅館管理、餐飲管理、休閒事業管理、幼兒保育等，培養服務產業人才。
        *   **設計學院：** 設有空間設計、多媒體與遊戲發展、時尚造型設計等系所。
    *   提供學士、碩士等學位課程。

5.  **學校發展與展望：**
    *   積極推動國際化，與多所國外大學建立合作關係，提供交換學生及雙聯學位機會。
    *   持續引進先進設備與師資，強化教學品質與研發能量，期許成為技職教育的典範大學。

總體而言，明新科技大學是一所具備深厚歷史底蘊、緊密結合產業脈動，並致力於培養具備即戰力之專

In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是：**劉國偉校長**。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:40:19] "GET / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:40:54] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:41:16] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[]}


...

INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:45:49] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595637029980078523","quoteToken":"qOSTXkkyzXd5rxjf-Ps_rp_NNhBMmFXmGGlG3x7H3UyYksqKXowzUIXU5EiPwEEULMw7Jm_JL3KkDGC_ZAO1fdqSvTP2j96L0tbDNJOQflft60sL8UxiNbfxeAeC8M2Qh_c2F_aXC2HNOjYjJuacwA","markAsReadToken":"bQGFHvMbg7mToDRcyRzeMPGSbGN75wnWwQD6pPT7EJRrBApLQny0_mJxNnA4p9oszi1w1kzUGBmjg2WJM41zUhNwzRvuocgLImoyM8jdxAWF2waze6l-0vzlgENFh7w6UJmPvSElMEahVcrgGljs_8GbjgEp9GjKy9AxVKN_2HLf2qmA-o3hQMSxOCkQWcnij3UBu8iCNcAXHTZUYbcVUA","text":"AI who are you"},"webhookEventId":"01KEE9C5G21Q35MD2SMNBJ533G","deliveryContext":{"isRedelivery":false},"timestamp":1767858639877,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"fdf917f80fc740f0a4b1b8b1f3d240ac","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:50:44] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595637066823107244","quoteToken":"horxFlW72mVEo9J1WZBRVbSpbql-KXWje87MZPQiZSNNuTE6LItuyoJYk9eYOwLzb2I9pK2nqYYyAazS143i9O7a6K7pl8RfDjbX4-wz5ofTX1DpIPVuAGxvdUXgDhC8Vns5WSY5tfLm94XFznuKfw","markAsReadToken":"uU5z29evmPpBS3Z4chD7xeiczR1wx8XJFP76OYSv48ornrP3dv2HNfm_ialju0xGrceJ1ZDxGdK_eLc75uEiCVxNRzT8tEZVvHuBZHhNGoXBtIJ-plg1NAuw7kid3HliW-ngE8OquGBbHDJe7Sv2vG4_CAcbm9ujSlCwxTyRrXA04i4szxa5UGD4uD6Ka0WheR8MttMxHtj0dLPE2Q84yw","text":"AI 校長是誰？"},"webhookEventId":"01KEE9CTYP59H3PFD41H9JRQMA","deliveryContext":{"isRedelivery":false},"timestamp":1767858661975,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"2c364fa540f8419a935fbb15c63902ce","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 07:51:06] "POST / HTTP/1.1" 200 -
